<a href="https://colab.research.google.com/github/xi1224-pan/ml-1/blob/main/%E6%96%B0%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install numpy==1.24.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.3 requires numpy>=1.26, but you have numpy 1.24.4 which is 

In [2]:
!pip uninstall -y transformers


Found existing installation: transformers 4.40.0
Uninstalling transformers-4.40.0:
  Successfully uninstalled transformers-4.40.0


In [3]:
!pip uninstall -y huggingface_hub

Found existing installation: huggingface-hub 0.32.2
Uninstalling huggingface-hub-0.32.2:
  Successfully uninstalled huggingface-hub-0.32.2


In [4]:
import shutil
shutil.rmtree('/root/.cache/huggingface', ignore_errors=True)
shutil.rmtree('/root/.cache/torch', ignore_errors=True)

In [5]:
!pip install --upgrade --force-reinstall transformers

  Using cached transformers-4.52.3-py3-none-any.whl.metadata (40 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.32.2-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Usi

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,  EarlyStoppingCallback
import torch
from google.colab import drive
drive.mount('/content/drive')

# 1. 載入資料
df1 = pd.read_csv("/content/drive/MyDrive/奧德賽標註.csv")
df2 = pd.read_csv("/content/drive/MyDrive/反諷標註.csv")
df3 = pd.read_csv("/content/drive/MyDrive/新評論.csv")
df = pd.concat([df1, df2, df3], ignore_index=True)
# 2. 分割資料集
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

# 3. 轉換為 Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 4. 載入 tokenizer 與模型
model_name = "hfl/chinese-roberta-wwm-ext"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# 5. Tokenize
def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset = val_dataset.map(tokenize_fn, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# 6. 訓練參數
training_args = TrainingArguments(
    output_dir="./mario_model",
    logging_dir="/content/drive/MyDrive/logs",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# 7. 評估函式
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    return {"accuracy": accuracy_score(labels, preds)}

# 8. 建立 Trainer 並開始訓練
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

# 9. 儲存模型
model.save_pretrained("/content/drive/MyDrive/mario_sentiment_新模型")
tokenizer.save_pretrained("/content/drive/MyDrive/mario_sentiment_新模型")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,1.282700,0.938821,0.638889
2,0.670400,0.419695,0.888889
3,0.171100,0.274266,0.888889
4,0.035800,0.407377,0.833333
5,0.015800,0.286489,0.888889


('/content/drive/MyDrive/mario_sentiment_新模型/tokenizer_config.json',
 '/content/drive/MyDrive/mario_sentiment_新模型/special_tokens_map.json',
 '/content/drive/MyDrive/mario_sentiment_新模型/vocab.txt',
 '/content/drive/MyDrive/mario_sentiment_新模型/added_tokens.json')